In [ ]:
import pandas as pd
import polars as pl
import numpy as np
from random import randint
import random
import time
from rtsvg import *
rt = RACETrack()
rt.co_mgr.str_to_color_lu['pandas'] = '#ffa0a0' # reddish
rt.co_mgr.str_to_color_lu['polars'] = '#a0a0ff' # bluish

In [ ]:
# Create A Randomized Dataset
def createData(sz=1000):
    strs_20_set = ['ant', 'bat', 'cat', 'dog', 'elephant', 'fox', 'gecko', 'horse', 'izard', 'jackal', 'kite', 'lizard', 'mouse', 
                   'ostrich', 'parrot', 'rabbit', 'snake', 'turtle', 'zebra']
    strs_5_set  = ['blue', 'red', 'green', 'orange', 'yellow']
    strs_20, nums_small, nums_large, dats, strs_5, nums_float = [], [], [], [], [], []
    for i in range(sz):
        strs_20.append(strs_20_set[randint(0,len(strs_20_set)-1)])
        nums_small.append(randint(0, 9))
        nums_large.append(randint(0, 99)), nums_float.append(random.random())
        year, month, day, hour, minute, second  = randint(1900, 2020), randint(1, 12), randint(1, 28), randint(0, 23), randint(0, 59), randint(0, 59)
        dats.append(f'{year:04}-{month:02}-{day:02} {hour:02}:{minute:02}:{second:02}')
        strs_5.append(strs_5_set[randint(0,4)])
    df_pd = pd.DataFrame({'strs_20':strs_20, 'nums_small':nums_small, 'nums_large':nums_large, 'nums_float':nums_float, 'dats':dats, 'strs_5':strs_5})
    df_pd = rt.columnsAreTimestamps(df_pd, 'dats')
    df_pl = pl.DataFrame(df_pd)
    return df_pd, df_pl

# Create Datasets Of Various Sizes
pds, pls, szs = {}, {}, []
for i in range(100000,1000000,100000):
    szs.append(i)
df_pd, df_pl = createData(max(szs))
for sz in szs:
    df_pl_sample = df_pl.sample(sz) if (sz != len(df_pl)) else df_pl
    df_pd_sample = df_pd.sample(sz) if (sz != len(df_pd)) else df_pd
    pds[sz] = df_pd_sample
    pls[sz] = df_pl_sample
    
# Time Specific Method
def timeMethod(fn, params, component, count_by, samples=5):
    xs, ys, types, components, count_bys, groups, svg_lengths = [], [], [], [], [], [], []
    for sz in pds.keys():
        accum_time = 0.0
        for _sample_ in range(samples):
            ts0 = time.time()
            _pandas_str_ = fn(pds[sz], **params)._repr_svg_()
            _pandas_str_len_ = len(_pandas_str_)
            ts1 = time.time()
            accum_time += (ts1 - ts0)
        pd_time = accum_time/samples
        accum_time = 0.0
        for _sample_ in range(samples):
            ts0 = time.time()
            _polars_str_ = fn(pls[sz], **params)._repr_svg_()
            _polars_str_len_ = len(_polars_str_)
            ts1 = time.time()
            accum_time += (ts1 - ts0)
        pl_time = accum_time/samples
        xs.append(sz), ys.append(pd_time), types.append('pandas'), components.append(component), count_bys.append(count_by), groups.append('a'), svg_lengths.append(_pandas_str_len_)
        xs.append(sz), ys.append(pl_time), types.append('polars'), components.append(component), count_bys.append(count_by), groups.append('b'), svg_lengths.append(_polars_str_len_)
    return pd.DataFrame({'size':xs, 'time':ys, 'type':types, 'component':components, 'count_by':count_bys, 'group':groups, 'svg_length':svg_lengths})
xy_params = {'x_field':'size', 'y_field':'time', 'line_groupby_field':'group', 'line_groupby_w':3, 'dot_size':None, 'color_by':'type', 'w':512}
all_dfs  = []

In [ ]:
df  = timeMethod(rt.histogram, {'bin_by':'strs_20'},                          'histograms', 'rows')
df2 = timeMethod(rt.histogram, {'bin_by':'strs_20', 'count_by':'nums_large'}, 'histograms', 'ints')
df3 = timeMethod(rt.histogram, {'bin_by':'strs_20', 'count_by':'nums_float'}, 'histograms', 'floats')
all_dfs.extend([df, df2, df3])
rt.tile([rt.xy(df, **xy_params), rt.xy(df2, **xy_params), rt.xy(df3, **xy_params)])

In [ ]:
df  = timeMethod(rt.temporalBarChart, {},                        'temporalBarChart', 'rows')
df2 = timeMethod(rt.temporalBarChart, {'count_by':'nums_large'}, 'temporalBarChart', 'ints')
df3 = timeMethod(rt.temporalBarChart, {'count_by':'nums_float'}, 'temporalBarChart', 'floats')
all_dfs.extend([df, df2, df3])
rt.tile([rt.xy(df, **xy_params), rt.xy(df2, **xy_params), rt.xy(df3, **xy_params)])

In [ ]:
df  = timeMethod(rt.xy, {'x_field':'nums_float', 'y_field':'nums_large'},                      'xy', 'default')
df2 = timeMethod(rt.xy, {'x_field':'dats',       'y_field':'nums_large'},                      'xy', 'time')
df3 = timeMethod(rt.xy, {'x_field':'nums_float', 'y_field':'nums_large', 'color_by':'strs_5'}, 'xy', 'default_color')
df4 = timeMethod(rt.xy, {'x_field':'dats',       'y_field':'nums_large', 'color_by':'strs_5'}, 'xy', 'time_color')
all_dfs.extend([df,df2,df3,df4])
rt.tile([rt.xy(df, **xy_params), rt.xy(df2, **xy_params), rt.xy(df3, **xy_params), rt.xy(df4, **xy_params)])

In [ ]:
df  = timeMethod(rt.periodicBarChart, {},                        'periodicBarChart', 'rows')
df2 = timeMethod(rt.periodicBarChart, {'count_by':'nums_large'}, 'periodicBarChart', 'ints')
df3 = timeMethod(rt.periodicBarChart, {'count_by':'nums_float'}, 'periodicBarChart', 'floats')
all_dfs.extend([df, df2, df3])
rt.tile([rt.xy(df, **xy_params), rt.xy(df2, **xy_params), rt.xy(df3, **xy_params)])

In [ ]:
relates = [('strs_20','strs_5')]
df = timeMethod(rt.linkNode, {'relationships':relates}, 'linkNode', 'default')
all_dfs.append(df)
rt.xy(df, **xy_params)

In [ ]:
relates = [('strs_20','strs_5')]
df = timeMethod(rt.chordDiagram, {'relationships':relates}, 'chordDiagram', 'default')
all_dfs.append(df)
rt.xy(df, **xy_params)

In [ ]:
rt.smallMultiples(pd.concat(all_dfs), category_by=['component','count_by'], grid_view=False, show_df_multiple=False, sm_type='xy', sort_by='field', sort_by_field='time',
                  sm_params={'x_field':'size', 'y_field':'time', 'line_groupby_field':'group', 'line_groupby_w':3, 'dot_size':'medium', 'color_by':'type'},
                  y_axis_independent=False, w_sm_override=256, h_sm_override=140, w=1400)

## 2024-02-11 amd7900x 96gb
### red = pandas, blue = polars
'<svg id="smallMultiples_47440" x="0" y="0" width="1292" height="474" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="473" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_47440_0" x="2" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 46,93 75,80 104,68 134,55 163,42 192,31 221,15 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,116 46,111 75,105 104,102 134,97 163,92 192,90 221,84 251,82 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="108" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="93" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="111" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="80" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="105" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="68" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="102" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="55" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="97" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="42" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="92" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="31" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="90" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="15" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="84" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="82" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">chordDiagram|default</text><svg id="smallMultiples_47440_1" x="260" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,112 46,103 75,94 104,86 134,79 163,69 192,61 221,54 251,46 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,119 163,119 192,118 221,118 251,118 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="112" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="103" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="94" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="79" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="69" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="61" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="54" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="46" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|rows</text><svg id="smallMultiples_47440_2" x="518" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,112 46,104 75,94 104,86 134,78 163,71 192,62 221,55 251,47 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,119 134,119 163,119 192,118 221,118 251,118 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="112" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="104" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="94" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="78" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="71" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="62" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="55" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="47" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|ints</text><svg id="smallMultiples_47440_3" x="776" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,112 46,103 75,95 104,86 134,79 163,71 192,62 221,56 251,48 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,119 134,119 163,119 192,118 221,118 251,118 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="112" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="103" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="95" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="79" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="71" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="62" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="56" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="48" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|floats</text><svg id="smallMultiples_47440_4" x="1034" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,104 46,103 75,102 104,102 134,102 163,101 192,101 221,100 251,100 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,115 46,115 75,114 104,114 134,114 163,113 192,113 221,112 251,112 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="104" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="103" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="101" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="101" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="100" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="100" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">xy|time_color</text><svg id="smallMultiples_47440_5" x="2" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,105 46,104 75,103 104,102 134,103 163,102 192,101 221,100 251,100 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,115 46,115 75,114 104,114 134,114 163,113 192,113 221,113 251,112 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="105" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="104" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="103" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="103" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="101" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="100" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="100" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|default_color</text><svg id="smallMultiples_47440_6" x="260" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,111 46,110 75,110 104,110 134,110 163,109 192,109 221,109 251,108 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,118 46,118 75,118 104,118 134,118 163,118 192,118 221,118 251,118 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="111" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="109" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="109" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="109" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="108" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|time</text><svg id="smallMultiples_47440_7" x="518" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,111 46,111 75,111 104,110 134,110 163,110 192,109 221,109 251,108 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,119 46,118 75,118 104,118 134,118 163,118 192,118 221,118 251,118 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="111" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="111" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="111" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="109" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="109" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="108" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|default</text><svg id="smallMultiples_47440_8" x="776" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,119 75,118 104,117 134,117 163,116 192,115 221,113 251,113 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,119 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="116" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="113" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="113" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|floats</text><svg id="smallMultiples_47440_9" x="1034" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,119 75,118 104,117 134,117 163,116 192,114 221,114 251,113 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,119 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="116" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="114" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="114" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="113" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|ints</text><svg id="smallMultiples_47440_10" x="2" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,119 46,119 75,118 104,118 134,117 163,116 192,115 221,114 251,114 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,119 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="116" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="114" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="114" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|rows</text><svg id="smallMultiples_47440_11" x="260" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,119 75,119 104,118 134,117 163,117 192,116 221,115 251,114 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,119 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="116" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="114" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">linkNode|default</text><svg id="smallMultiples_47440_12" x="518" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,120 75,120 104,119 134,119 163,118 192,118 221,117 251,117 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,120 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|rows</text><svg id="smallMultiples_47440_13" x="776" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,120 75,120 104,119 134,119 163,119 192,118 221,118 251,117 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,120 46,120 75,120 104,120 134,120 163,120 192,119 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|floats</text><svg id="smallMultiples_47440_14" x="1034" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,120 46,120 75,120 104,119 134,119 163,119 192,118 221,118 251,117 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,120 104,120 134,120 163,120 192,120 221,119 251,119 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="120" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#4988b6" stroke="#4988b6" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="119" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="118" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|ints</text><rect width="1291" height="473" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>'

In [ ]:
rt.smallMultiples(pd.concat(all_dfs), category_by=['component','count_by'], grid_view=False, show_df_multiple=False, sm_type='xy', sort_by='field', sort_by_field='time',
                  sm_params={'x_field':'size', 'y_field':'time', 'line_groupby_field':'group', 'line_groupby_w':3, 'dot_size':'medium', 'color_by':'type'},
                  y_axis_independent=True, w_sm_override=256, h_sm_override=140, w=1400)

## 2024-02-11 amd7900x 96gb
### red = pandas, blue = polars
'<svg id="smallMultiples_50617" x="0" y="0" width="1292" height="474" xmlns="http://www.w3.org/2000/svg"><rect width="1291" height="473" x="0" y="0" fill="#ffffff" stroke="#ffffff" /><svg id="smallMultiples_50617_0" x="2" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,112 46,97 75,83 104,71 134,57 163,44 192,32 221,15 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,115 75,110 104,106 134,102 163,96 192,93 221,88 251,85 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="112" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="97" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="83" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="110" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="71" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="106" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="57" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="102" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="44" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="96" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="32" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="93" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="15" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="88" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="85" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.14</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">3.20</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">chordDiagram|default</text><svg id="smallMultiples_50617_1" x="260" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 46,94 75,79 104,66 134,54 163,40 192,27 221,15 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,120 104,119 134,119 163,118 192,118 221,117 251,117 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="108" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="94" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="79" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="66" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="54" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="40" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="27" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="15" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">2.03</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|rows</text><svg id="smallMultiples_50617_2" x="518" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,108 46,94 75,79 104,66 134,52 163,41 192,27 221,15 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,120 104,119 134,119 163,118 192,118 221,117 251,117 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="108" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="94" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="79" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="66" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="52" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="41" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="27" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="15" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">1.99</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|ints</text><svg id="smallMultiples_50617_3" x="776" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,107 46,93 75,79 104,64 134,52 163,40 192,25 221,14 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,120 104,119 134,119 163,118 192,118 221,117 251,116 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="107" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="93" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="79" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="64" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="52" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="40" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="25" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="14" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">1.96</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">periodicBarChart|floats</text><svg id="smallMultiples_50617_4" x="1034" y="2" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,37 46,29 75,24 104,22 134,22 163,11 192,12 221,3 251,4 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,117 75,115 104,112 134,109 163,106 192,104 221,101 251,99 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="37" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="29" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="24" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="22" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="22" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="109" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="11" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="106" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="12" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="104" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="101" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="4" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="99" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.15</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.57</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="154" font-family="Times" fill="#000000" font-size="14px">xy|time_color</text><svg id="smallMultiples_50617_5" x="2" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,39 46,34 75,25 104,21 134,26 163,16 192,11 221,6 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,117 75,114 104,112 134,109 163,106 192,103 221,101 251,98 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="39" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="34" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="25" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="21" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="26" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="109" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="16" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="106" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="11" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="103" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="6" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="101" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="98" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.15</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.56</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|default_color</text><svg id="smallMultiples_50617_6" x="260" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,29 46,20 75,25 104,22 134,18 163,14 192,10 221,6 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,120 104,119 134,118 163,118 192,117 221,117 251,117 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="29" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="20" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="25" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="22" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="18" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="14" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="10" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="6" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.06</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.33</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|time</text><svg id="smallMultiples_50617_7" x="518" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,31 46,36 75,30 104,27 134,22 163,19 192,15 221,10 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,119 75,119 104,118 134,118 163,117 192,116 221,116 251,115 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="31" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="36" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="30" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="27" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="22" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="19" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="15" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="10" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="115" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.06</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.34</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">xy|default</text><svg id="smallMultiples_50617_8" x="776" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,111 46,99 75,87 104,74 134,62 163,44 192,28 221,10 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,119 75,117 104,116 134,112 163,110 192,109 221,107 251,105 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="111" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="99" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="87" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="74" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="62" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="44" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="110" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="28" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="109" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="10" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="107" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="105" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.21</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|floats</text><svg id="smallMultiples_50617_9" x="1034" y="160" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,110 46,100 75,86 104,75 134,61 163,44 192,25 221,12 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,119 75,118 104,116 134,112 163,110 192,108 221,106 251,104 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="110" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="100" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="75" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="61" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="44" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="110" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="25" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="108" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="12" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="106" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="104" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.21</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="312" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|ints</text><svg id="smallMultiples_50617_10" x="2" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,101 46,99 75,86 104,73 134,59 163,43 192,30 221,13 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,120 75,118 104,116 134,112 163,110 192,107 221,105 251,103 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="101" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="99" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="120" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="118" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="73" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="59" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="43" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="110" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="30" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="107" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="13" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="105" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="103" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.19</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="130.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">temporalBarChart|rows</text><svg id="smallMultiples_50617_11" x="260" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,117 46,106 75,93 104,78 134,62 163,57 192,46 221,17 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,119 75,116 104,114 134,110 163,107 192,104 221,100 251,95 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="117" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="106" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="119" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="93" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="78" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="114" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="62" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="110" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="57" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="107" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="46" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="104" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="17" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="100" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="95" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.01</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.18</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="388.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">linkNode|default</text><svg id="smallMultiples_50617_12" x="518" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,115 46,105 75,93 104,83 134,67 163,51 192,32 221,17 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,117 75,113 104,106 134,100 163,96 192,89 221,85 251,77 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="105" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="93" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="83" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="106" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="67" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="100" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="51" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="96" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="32" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="89" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="17" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="85" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="77" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.10</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="646.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|rows</text><svg id="smallMultiples_50617_13" x="776" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,115 46,104 75,96 104,86 134,71 163,56 192,35 221,25 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,116 75,113 104,104 134,99 163,94 192,86 221,81 251,73 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="104" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="116" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="96" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="113" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="86" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="104" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="71" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="99" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="56" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="94" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="35" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="86" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="25" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="81" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="73" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.10</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="904.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|floats</text><svg id="smallMultiples_50617_14" x="1034" y="318" width="256" height="140" xmlns="http://www.w3.org/2000/svg"><rect width="255" height="139" x="0" y="0" fill="#ffffff" fill-opacity="1.0" stroke="#ffffff" stroke-opacity="1.0" /><path d="M 17 123 L 17 3 L 251 3 L 251 123 Z" stroke="#101010" stroke-width=".4" fill="none" /><polyline points="17,115 46,102 75,95 104,82 134,65 163,49 192,27 221,16 251,3 " stroke="#ffa0a0" stroke-width="3" fill="none" /><polyline points="17,121 46,117 75,112 104,104 134,97 163,92 192,85 221,79 251,70 " stroke="#a0a0ff" stroke-width="3" fill="none" /><circle cx="17" cy="115" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="17" cy="121" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="102" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="46" cy="117" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="95" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="75" cy="112" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="82" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="104" cy="104" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="65" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="134" cy="97" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="49" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="163" cy="92" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="27" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="192" cy="85" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="16" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="221" cy="79" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="3" r="2" fill="#ffa0a0" stroke="#ffa0a0" fill-opacity="1.0" stroke-opacity="1.0" /><circle cx="251" cy="70" r="2" fill="#a0a0ff" stroke="#a0a0ff" fill-opacity="1.0" stroke-opacity="1.0" /><text x="17" text-anchor="start" y="137" font-family="Times" fill="#000000" font-size="12px">100000</text><text x="251" text-anchor="end" y="137" font-family="Times" fill="#000000" font-size="12px">900000</text><text x="134.0" text-anchor="middle" y="137" font-family="Times" fill="#000000" font-size="12px">size</text><text x="13" text-anchor="start" y="121" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,121)">0.00</text><text x="13" text-anchor="end" y="3" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,3)">0.09</text><text x="13" text-anchor="middle" y="62.0" font-family="Times" fill="#000000" font-size="12px" transform="rotate(-90,13,62.0)">time</text><rect width="255" height="140" x="0" y="0" fill-opacity="0.0" fill="none" stroke="#000000" /></svg><text x="1162.0" text-anchor="middle" y="470" font-family="Times" fill="#000000" font-size="14px">histograms|ints</text><rect width="1291" height="473" x="0" y="0" fill="none" fill-opacity="0.0" stroke="#000000" /></svg>'

In [ ]:
rt.smallMultiples(pd.concat(all_dfs), category_by=['component','count_by'], grid_view=False, show_df_multiple=False, sm_type='xy', sort_by='field', sort_by_field='svg_length',
                  sm_params={'x_field':'size', 'y_field':'svg_length', 'line_groupby_field':'group', 'line_groupby_w':3, 'dot_size':'medium', 'color_by':'type'},
                  y_axis_independent=True, w_sm_override=256, h_sm_override=140, w=1400)